In [1]:
import numpy as np
import pandas as pd
import pathlib
import sqlite3

In [ ]:
BASE_FOLDER = pathlib.Path(".\\data\\33\\Phase")
DATABASE_FILE = BASE_FOLDER / "33_phase_all.sqlite3"
DATABASE_TRAIN = BASE_FOLDER / "33_phase_separated.sqlite3"
CONTAINING_FOLDERS = list(BASE_FOLDER.glob("./**"))
print(CONTAINING_FOLDERS)

In [3]:
# Extracts all data from csv file to a dataframe, adds file name to dataframe
def csv_extractor(file, index_start=None):
    data = pd.read_csv(file, header=0)
    new_column = [file.parts[-1][:-4]] * len(data)
    data.insert(loc=0, column="file_name", value=new_column)
    # set rigth index if dealing with multiple csvs
    if index_start:
        data.index = range(index_start, index_start + len(data))
    return data

def create_connect_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except sqlite3.Error as e:
        print(e)
    if conn:
            return conn
        
def create_table(db_conn, table_name, params_sql):
    create_table_sql = f"CREATE TABLE IF NOT EXISTS '{table_name}' (" + params_sql +");"
    try:
        curs = db_conn.cursor()
        curs.execute(create_table_sql)
        print(f"table {table_name} created.")
    except sqlite3.Error as e:
        print(e)
        
def write_to_db(db_conn, data, table_name):
    data.to_sql(table_name, db_conn, if_exists="append", index=True, index_label="myindex")
    print("rows written")
    

# ther has to be an "index" column in database    
def read_from_db(db_conn, table_name=None, what="*", condition=None):
    request_sql = f"SELECT {what}"
    if table_name:
        request_sql += f" FROM '{table_name}'"
    if condition:
        request_sql +=  " WHERE " + condition
    print(request_sql)
    read_data = pd.read_sql(request_sql, db_conn, index_col="myindex")
    return read_data

def create_phase_123():
    table_name = "dataset_all"
    table_params = "'myindex' INT PRIMARY KEY, 'folder_name' TEXT, 'file_name' TEXT"
    for i in range(1, 123):
        table_params += f", 'phase{i}' FLOAT"
    table_params += ", 'label' INT" 
    return table_params

def create_voltage_123():
    table_name = "dataset_all"
    table_params = "'myindex' INT PRIMARY KEY, 'folder_name' TEXT, 'file_name' TEXT"
    for i in range(1, 123):
        table_params += f", 'voltage{i}' FLOAT"
    table_params += ", 'label' INT"
    return table_params

def create_voltage_33():
    table_name = "dataset_all"
    table_params = "'myindex' INT PRIMARY KEY, 'folder_name' TEXT, 'file_name' TEXT"
    for i in range(1, 33):
        table_params += f", 'voltage{i}' FLOAT"
    table_params += ", 'label' INT"
    return table_params

def create_phase_33():
    table_name = "dataset_all"
    table_params = "'myindex' INT PRIMARY KEY, 'folder_name' TEXT, 'file_name' TEXT"
    for i in range(1, 33):
        table_params += f", 'phase{i}' FLOAT"
    table_params += ", 'label' INT"
    return table_params

In [30]:
all_connection = create_connect_database(DATABASE_FILE)
table_name = "dataset_all"
table_params = create_phase_33()   # update with database
create_table(all_connection, table_name=table_name, params_sql=table_params)
index_start = 0
for k in range(len(CONTAINING_FOLDERS)):
    files = list(CONTAINING_FOLDERS[k].glob("./*.csv"))
    for file in files:
        data_pd = csv_extractor(file, index_start=index_start)
        folder_name_column = [file.parts[-2]] * len(data_pd)
        data_pd.insert(loc=0, column="folder_name", value=folder_name_column)
        write_to_db(all_connection, data_pd, table_name=table_name)
        index_start += len(data_pd)

table dataset_all created.
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows written
rows writte

In [49]:
read_data = read_from_db(all_connection, "dataset_all", "*", condition = "label=1515")
print(read_data)

SELECT * FROM 'dataset_all' WHERE label=1515
             folder_name file_name  voltage1  voltage2  voltage3  voltage4  \
myindex                                                                      
22600    Case1DecVoltage         5    1.0387    1.0334    1.0332    1.0332   
22601    Case1DecVoltage         5    1.0385    1.0331    1.0329    1.0328   
22602    Case1DecVoltage         5    1.0386    1.0331    1.0330    1.0329   
22603    Case1DecVoltage         5    1.0386    1.0332    1.0330    1.0329   
22604    Case1DecVoltage         5    1.0386    1.0333    1.0331    1.0331   
...                  ...       ...       ...       ...       ...       ...   
224443   Case4IncVoltage         5    1.0386    1.0332    1.0331    1.0330   
224444   Case4IncVoltage         5    1.0389    1.0337    1.0336    1.0335   
224445   Case4IncVoltage         5    1.0396    1.0352    1.0351    1.0350   
224446   Case4IncVoltage         5    1.0400    1.0361    1.0359    1.0358   
224447   Case4IncVo

In [29]:
'''Write headers to files'''
for k in range(1, len(CONTAINING_FOLDERS)):
    files = list(CONTAINING_FOLDERS[k].glob("./*.csv"))
    for file in files:
        inner_data = csv_extractor(file, index_start=None)
        table_params = []
        for i in range(1, 33):
            table_params.append(f"phase{i}")
        table_params.append("label")
        data = pd.read_csv(file, names=table_params)
        file_name_column = [file.parts[-1][:-4]] * len(data)
        #data.to_csv(file, header=True, index=False)             #hazardous zone


# Seperating train, validation, test

In [32]:
separation_connection = create_connect_database(DATABASE_TRAIN)
table_names = ["Train", "Validation", "Test"]
table_name = "dataset_all"
table_params = create_phase_33()   # update with database
for name in table_names:            
    create_table(separation_connection, table_name=name, params_sql=table_params)

table Train created.
table Validation created.
table Test created.


In [33]:
all_connection = create_connect_database(DATABASE_FILE)
all_data = read_from_db(all_connection, "dataset_all", "*", condition = None)

SELECT * FROM 'dataset_all'


Zeros are the same in each folder. so, here is how we seperate validation & test data:
there are 48 labeled data in each file in each folder. we take 15 for validation and 33 for testing.
after label picking from every folder is finished, we add 330 number of zero to test set and 150 to validation set and add the rest to train set. 

In [36]:
def dataset_splitter(all_data, n_label_validation=15, n_zero_validation=75, n_label_test=33, n_zero_test=165):
    index_start_train, index_start_validation, index_start_test = 0, 0, 0

    unique_lables = all_data["label"].unique()
    folder_names = all_data["folder_name"].unique()
    file_names = all_data["file_name"].unique()
    print(file_names)

    train_set = pd.DataFrame(columns=all_data.columns)
    validation_set = pd.DataFrame(columns=all_data.columns)
    test_set = pd.DataFrame(columns=all_data.columns)

    for folder in folder_names:
        all_in_folder = all_data[all_data.folder_name == folder]
        print("all in folder ", folder, len(all_in_folder))
        validation_temp = pd.DataFrame(columns=all_data.columns)
        test_temp = pd.DataFrame(columns=all_data.columns)
        for name in file_names:
            #if folder == "Case1DecPhase" and name == "1":   # uncomment this if doing Dataset folder-phase
                #print("condition met")
                #continue
            all_in_name = all_in_folder[all_in_folder.file_name == name]

            labeled = all_in_name[all_in_name.label != 0]
            zeros = all_in_name[all_in_name.label == 0]
            print("len all in file ", name, len(all_in_name), "zeros", len(zeros))

            validation_lbl = labeled.sample(n=n_label_validation)
            validation_temp = validation_temp.append(validation_lbl)
            labeled.drop(axis=0, index=validation_lbl.index, inplace=True)

            test_temp = test_temp.append(labeled)

        validation_set = validation_set.append(validation_temp)
        validation_zeros = zeros.sample(n=n_zero_validation)
        validation_set = validation_set.append(validation_zeros)
        zeros.drop(axis=0, index=validation_zeros.index, inplace=True)

        test_set = test_set.append(test_temp)
        test_zeros = zeros.sample(n=n_zero_test)
        test_set = test_set.append(test_zeros)
        zeros.drop(axis=0, index=test_zeros.index, inplace=True)

        train_set = train_set.append(zeros)

    train_set.reset_index(drop=True, inplace=True)
    validation_set.reset_index(drop=True, inplace=True)
    test_set.reset_index(drop=True, inplace=True)
    return train_set, validation_set, test_set

In [37]:
train_set, validation_set, test_set = dataset_splitter(all_data, n_label_validation = 15, n_zero_validation = 75, n_label_test = 33, n_zero_test = 165)

['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '4' '5' '6'
 '7' '8' '9']
all in folder  Case1DecPhase 47520
len all in file  1 1440 zeros 1392
len all in file  10 1440 zeros 1392
len all in file  11 1440 zeros 1392
len all in file  12 1440 zeros 1392
len all in file  13 1440 zeros 1392
len all in file  14 1440 zeros 1392
len all in file  15 1440 zeros 1392
len all in file  16 1440 zeros 1392
len all in file  17 1440 zeros 1392
len all in file  18 1440 zeros 1392
len all in file  19 1440 zeros 1392
len all in file  2 1440 zeros 1392
len all in file  20 1440 zeros 1392
len all in file  21 1440 zeros 1392
len all in file  22 1440 zeros 1392
len all in file  23 1440 zeros 1392
len all in file  24 1440 zeros 1392
len all in file  25 1440 zeros 1392
len all in file  26 1440 zeros 1392
len all in file  27 1440 zeros 1392
len all in file  28 1440 zeros 1392
len all in file  29 1440 zeros 1392
len all in file

In [38]:
print(len(train_set), len(validation_set), len(test_set))

9216 4560 10032


In [39]:
write_to_db(separation_connection, train_set, table_name="Train")
write_to_db(separation_connection, validation_set, table_name="Validation")
write_to_db(separation_connection, test_set, table_name="Test")

rows written
rows written
rows written
